In [2]:
# Import dependencie
import pandas as pd

# Load file 
school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

# Read the data and store into pandas dataframes
school_data_df = pd.read_csv(school_data)
student_data_df = pd.read_csv(student_data)

# Combine the data into a single dataset
complete_data_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
complete_data_df.head() 

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [3]:
# Calculate total schools 
total_schools = len(complete_data_df["school_name"].unique())

# Calculate total students 
total_students = complete_data_df["Student ID"].count()

# Calculate total budget 
total_budget = sum(complete_data_df["budget"].unique())

# Calculate average math score 
average_math = complete_data_df["math_score"].mean()

# Calculate average reading score 
average_reading = complete_data_df["reading_score"].mean()

# Calculate the percentage of students that passed math
passing_math = sum(complete_data_df["math_score"] >=70) / total_students * 100

# Calculate the percentage of students that passed reading
passing_reading = sum(complete_data_df["reading_score"] >=70) / total_students * 100

# Calculate the percentage of students that passed math and reading
pass_math_df = complete_data_df.loc[complete_data_df["math_score"] >=70,:]
pass_both_df = pass_math_df.loc[pass_math_df["reading_score"] >=70,:]
overall_passing = len(pass_both_df) / total_students * 100

# Place all of the data found into a summary dataframe
district_summary_df = pd.DataFrame({"Total Schools": [total_schools],
                           "Total Students": [total_students],
                           "Total Budget": [total_budget],
                           "Average Math Score": [average_math],
                           "Average Reading Score": [average_reading],
                           "% Passing Math": [passing_math],
                           "% Passing Reading": [passing_reading],
                           "% Overall Passing": [overall_passing]})

# Format
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].astype(float).map("${:,.2f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [4]:
# Extract columns from school_data
school_df = school_data_df.loc[:, ["school_name", "type","budget"]]

# Find school name and school type 
ss_school_type = school_df.set_index("school_name")["type"]

# Find school name and school budget
ss_school_budget = school_df.set_index("school_name")["budget"]

In [5]:
# Remove unecessary columns from the single dataset and to create a new dataframe
reduced_df = complete_data_df[["school_name", "type", "budget", "math_score", "reading_score"]]

# Sort the dataframe based on the school name
sort_df = reduced_df.sort_values("school_name")

# Reset the index so that the index is now based on the school name
sort_df = sort_df.reset_index(drop=True)

## School Summary

In [6]:
# Calculate total students
ss_students = sort_df["school_name"].value_counts()

# Calculate per student budget 
ss_student_budget = ss_school_budget / ss_students 

# Calculate average math score
ss_groupby_math = sort_df.groupby("school_name")["math_score"].sum()
ss_average_math = ss_groupby_math / ss_students

# Caculate average reading score
ss_groupby_reading = sort_df.groupby("school_name")["reading_score"].sum()
ss_average_reading = ss_groupby_reading / ss_students

# Calculate the percentage of students that passed math
ss_pass_math_df = sort_df[sort_df["math_score"] >=70]
ss_passing_math = ss_pass_math_df.groupby("school_name")["math_score"].count() / ss_students * 100

# Calculate the percentage of students that passed reading
ss_pass_reading_df = sort_df[sort_df["reading_score"] >=70]
ss_passing_reading = ss_pass_reading_df.groupby("school_name")["reading_score"].count() / ss_students * 100

# Calculate the percentage of students that passed math and reading
ss_pass_both_df = ss_pass_math_df[ss_pass_math_df["reading_score"] >=70]
ss_overall_passing = ss_pass_both_df.groupby("school_name")["reading_score"].count() / ss_students * 100

# Place all of the data found into a summary dataframe
school_summary_df = pd.DataFrame({"School Type": ss_school_type,
                                  "Total Students": ss_students,
                                  "Total School Budget": ss_school_budget,
                                  "Per Student Budget": ss_student_budget,
                                  "Average Math Score": ss_average_math,
                                  "Average Reading Score": ss_average_reading,
                                  "% Passing Math": ss_passing_math,
                                  "% Passing Reading": ss_passing_reading,
                                  "% Overall Passing": ss_overall_passing})

# Format
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].astype(float).map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].astype(float).map("${:.2f}".format)

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

In [7]:
# Sort and display the top five performing schools by % overall passing.
ps_sorttop_df = school_summary_df.sort_values(["% Overall Passing"], ascending=False).head(5)
ps_sorttop_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [8]:
# Sort and display the five worst-performing schools by % overall passing.
ps_sortworst_df = school_summary_df.sort_values(["% Overall Passing"], ascending=True).head(5)
ps_sortworst_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [9]:
# Create a table that lists the school name, grade and average math score
grade_math_df = complete_data_df.groupby(["school_name", "grade"], as_index = False)["math_score"].mean()

# Looking only at grade 9
math_nine_df = grade_math_df.loc[grade_math_df["grade"] == "9th"]
mg_nine = math_nine_df.set_index("school_name")["math_score"]

# Looking only at grade 10
math_ten_df = grade_math_df.loc[grade_math_df["grade"] == "10th"]
mg_ten = math_ten_df.set_index("school_name")["math_score"] 

# Looking only at grade 11th
math_eleven_df = grade_math_df.loc[grade_math_df["grade"] == "11th"]
mg_eleven = math_eleven_df.set_index("school_name")["math_score"]

# Looking only at grade 12th
math_twelve_df = grade_math_df.loc[grade_math_df["grade"] == "12th"]
mg_twelve = math_twelve_df.set_index("school_name")["math_score"] 

# Place all of the data found into a summary dataframe
mg_summary_df = pd.DataFrame({"9th": mg_nine, "10th": mg_ten, "11th": mg_eleven, "12th": mg_twelve})
mg_summary_df.index.name = None
mg_summary_df

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Scores by Grade

In [10]:
# Create a table that lists the school name, grade and average reading score
grade_reading_df = complete_data_df.groupby(['school_name', 'grade'], as_index = False)['reading_score'].mean()

# Looking only at grade 9
reading_nine_df = grade_reading_df.loc[grade_reading_df["grade"] == "9th"]
rg_nine = reading_nine_df.set_index("school_name")["reading_score"]

# Looking only at grade 10
reading_ten_df = grade_reading_df.loc[grade_reading_df["grade"] == "10th"]
rg_ten = reading_ten_df.set_index("school_name")["reading_score"] 

# Looking only at grade 11th
reading_eleven_df = grade_reading_df.loc[grade_reading_df["grade"] == "11th"]
rg_eleven = reading_eleven_df.set_index("school_name")["reading_score"]

# Looking only at grade 12th
reading_twelve_df = grade_reading_df.loc[grade_reading_df["grade"] == "12th"]
rg_twelve = reading_twelve_df.set_index("school_name")["reading_score"] 

# Place all of the data found into a summary dataframe
rg_summary_df = pd.DataFrame({"9th": rg_nine, "10th": rg_ten, "11th": rg_eleven, "12th": rg_twelve})
rg_summary_df.index.name = None
rg_summary_df

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [11]:
# Remove unecessary columns from the single dataset and to create a new dataframe
ss_reduced_df = school_summary_df[["Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

# Create the bins in which data will be held
spending_bins = [0, 584, 629, 644, 675]

# Create the names for the bins
spending_names = ["<$584", "$585-629", "$630-644", "$645-675"]

# Place the data series into a new column inside of the dataframe
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(ss_student_budget, spending_bins, labels=spending_names, include_lowest=True)

# Calculate average math score
spending_math = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_math

# Calculate average reading score
spending_reading = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

# Calculate the percentage of students that passed math
spending_pm = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]

# Calculate the percentage of students that passed reading
spending_pr = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]

# Calculate the percentage of students that passed math and reading
spending_op = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

# Place all of the data found into a summary dataframe
spending_df = pd.DataFrame({"Average Math Score": spending_math,
                               "Average Reading Score": spending_reading, 
                               "% Passing Math": spending_pm,
                               "% Passing Reading": spending_pr,
                               "% Overall Passing": spending_op})

# Format
spending_df["Average Math Score"] = spending_df["Average Math Score"].map("{:.2f}".format)
spending_df["Average Reading Score"] = spending_df["Average Reading Score"].map("{:.2f}".format)
spending_df["% Passing Math"] = spending_df["% Passing Math"].map("{:.2f}".format)
spending_df["% Passing Reading"] = spending_df["% Passing Reading"].map("{:.2f}".format)
spending_df["% Overall Passing"] = spending_df["% Overall Passing"].map("{:.2f}".format)
spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size 

In [12]:
# Create the bins in which data will be held
size_bins = [0, 1000, 2000, 5000]

# Create the names for the bins
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Place the data series into a new column inside of the dataframe
school_summary_df["School Size"] = pd.cut(ss_students, size_bins, labels=size_names, include_lowest=True)

# Calculate average math score
size_math = school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]

# Calculate average reading score
size_reading = school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]

# Calculate the percentage of students that passed math
size_pm = school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]

# Calculate the percentage of students that passed reading
size_pr = school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]

# Calculate the percentage of students that passed math and reading
size_op = school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

# Place all of the data found into a summary dataframe
size_df = pd.DataFrame({"Average Math Score": size_math,
                               "Average Reading Score": size_reading, 
                               "% Passing Math": size_pm,
                               "% Passing Reading": size_pr,
                               "% Overall Passing": size_op})
size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [13]:
# Calculate average math score
type_math = school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]

# Calculate average reading score
type_reading = school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]

# Calculate the percentage of students that passed math
type_pm = school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]

# Calculate the percentage of students that passed reading
type_pr = school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]

# Calculate the percentage of students that passed math and reading
type_op = school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

# Place all of the data found into a summary dataframe
type_df = pd.DataFrame({"Average Math Score": type_math,
                        "Average Reading Score": type_reading, 
                        "% Passing Math": type_pm,
                        "% Passing Reading": type_pr,
                        "% Overall Passing": type_op})
type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


## Observable trends based on the data.

1) On average, students do better in reading than math.

2) Charter schools have less students than district schools. 

3) Students in charter schools do better than students in district schools. The top 5 performing schools are also charter schools while the bottom 5 performing schools are district schools.  

4) Students in small and medium schools are performing better than students in large schools. 

5) Students perfomance is best despite least spending budget per student while students performance is worst when spending budget per student is highest. 

Therefore, the school board and mayor can consider the following for the future:  
- If district schools are necessary, student intake needs to be reduced to about 2200 students.
- If district schools are not necessary, convert them to charter schools and establish more charter schools to accomodate all the students from district schools. 
- Limit spending budget per student to maximum $584.
- Spend on resources to improve students' math score.